In [1]:
import sys
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from osgeo import gdal
from osgeo import osr
from osgeo.gdalconst import *
import rasterio
#from rasterio.plot import show
driver = gdal.GetDriverByName('GTiff')
driver.Register()

coords_system = osr.SpatialReference()
coords_system.ImportFromEPSG(26919)

6

In [5]:
dir_path = os.path.realpath('./')
#print(dir_path)
cwd = os.getcwd()
#print(cwd)

hubbard_dir = (os.path.abspath(os.path.join(os.path.realpath('./'),'..')) + '/data/hubbard_brook/')
print(hubbard_dir)
dem = hubbard_dir + "hubbard_clipped.tif"
tanbeta = hubbard_dir + "hubbard_tanbeta.tif"
ldd = hubbard_dir + "hubbard_ldd.tif"


C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/hubbard_brook/


In [ ]:
def runDrainTheAquifer(elevationValues, waterStorageValues, wTableHeightValues, slope,
                       D, fi, Sy, Ks, m, psi_b, n_days, deltaT=1, decaying_transmissivity=True):
    '''
    elevationValues: array of elevation values of cells, i.e. DEM values
    waterStorageValues: array of volumetric water content of cells, in saturated conditions
    wTableHeightValues: array
    etc
    '''

    rows, cols = elevationValues.shape 
    n_steps = 24 * n_days
    Qdict = {}
    Sdict = {}
    Edict = {}
    ddict = {}
    runoffVol = 0
    outlet = [0,2]

    current_time = 0
    print("current time step: {}".format(current_time))
    Qdict[current_time] = np.zeros((rows, cols))
    Sdict[current_time] = waterStorageValues
    Edict[current_time] = elevationValues
    ddict[current_time] = wTableHeightValues
    print("water table height (d) is : {}".format(ddict[current_time]))
    print("water elevation is : {} m asl".format(Edict[current_time]))
    print("water storage (S) is : {}".format(Sdict[current_time]))
    print("cumulative runoff is {} m^3/s\n".format(runoffVol))

    current_time += deltaT
    while current_time < n_steps*deltaT:
        print("current time step: {}".format(current_time))
        Qdict[current_time] = np.zeros((rows, cols))
        Sdict[current_time] = np.zeros((rows, cols))
        Edict[current_time] = np.zeros((rows, cols))
        ddict[current_time] = np.zeros((rows, cols))
        for i in range(rows):
            for j in range(cols):
                previous_d = ddict[current_time-1][i,j]
                if decaying_transmissivity == True  and  previous_d < (D - psi_b):
                    T = (Ks*D) * np.exp(-1*(D-psi_b-previous_d)*(fi-Sy)/ m) #this is dynamic
                else:
                    T = Ks * previous_d
                Qout = T * resol * slope
                Qdict[current_time][i,j] = Qout
                if j>0:
                    Qin = Qdict[current_time][i,j-1]
                else:
                    Qin = 0

                if [i,j] == outlet:
                    runoffVol += Qout * deltaT

                deltaS = (Qin - Qout) * deltaT
                S = Sdict[current_time-1][i,j] + deltaS
                Sdict[current_time][i,j] = S

                deltad = deltaT*(Qin-Qout)/(Sy*resol*resol)
                d = ddict[current_time-1][i,j] + deltad
                if d > 0:
                    ddict[current_time][i,j] = d
                else:
                    ddict[current_time][i,j] = 0

                Edict[current_time][i,j] = Edict[0][i,j] - D + ddict[current_time][i,j]

        print("water table height (d) is : {}".format(ddict[current_time]))
        print("water elevation is : {} m asl".format(Edict[current_time]))
        print("water storage (S) is : {}".format(Sdict[current_time]))
        print("cumulative runoff is {} m^3\n".format(runoffVol))
        current_time += deltaT
    
    return ddict, Edict, Sdict, Qdict, runoffVol
    

D = 2  #thickness of unconfined aquifer, shallow groundwater
#parameters for sand  
Ks = 100 #micrometers per second
Ks = Ks * 3600 / 1000000 #meters per hour
decaying_transmissivity = False
m = 0.1 #this should be big number, so the exponent go to 0 (really? How big?)
psi_b = 0.1 #bubble pressure / capillary head, for sand
fi = 0.3
Sy = 0.15
n_days = 10
deltaT = 1


elevations_ds = gdal.Open(dem, GA_ReadOnly)
band = elevations_ds.GetRasterBand(1)
elevationValues = band.ReadAsArray(0, 0, elevations_ds.RasterXSize, elevations_ds.RasterYSize)
geotransform = elevations_ds.GetGeoTransform()
resol = geotransform[1]
resol = np.round(geotransform[1])

waterStorageFile = "storage.tif"
initial_storage = D*resol*resol*fi
waterStorageArray = np.array([[initial_storage, initial_storage, initial_storage]])
createRaster(waterStorageFile, waterStorageArray, resol)
waterStorage_ds = gdal.Open(waterStorageFile, GA_ReadOnly)
band = waterStorage_ds.GetRasterBand(1)
waterStorageValues = band.ReadAsArray(0, 0, waterStorage_ds.RasterXSize, waterStorage_ds.RasterYSize)

wTableHeightFile = "water_table_height.tif"
wTableHeightArray = np.array([[D, D, D]])
createRaster(wTableHeightFile, wTableHeightArray, resol)
wTableHeight_ds = gdal.Open(wTableHeightFile, GA_ReadOnly)
band = wTableHeight_ds.GetRasterBand(1)
wTableHeightValues = band.ReadAsArray(0, 0, wTableHeight_ds.RasterXSize, wTableHeight_ds.RasterYSize)

slopeFile = "slope.tif"
slope = np.absolute((elevationValues[0, 2] - elevationValues[0, 0]) / (2*resol))
#slopeArray = np.array([[slope, slope, slope]])
#createRaster(slopeFile, slopeArray, resol)

simul_results = runDrainTheAquifer(elevationValues, waterStorageValues, wTableHeightValues, slope,
                                   D, fi, Sy, Ks, m, psi_b, n_days, deltaT, decaying_transmissivity)

